# Tutorial IV: Constructing variational algorithms

Variational quantum algorithms are a broad set of methods which involve optimizing a parameterized quantum circuit applied to some initial state (called the "reference") in order to minimize a cost function defined with respect to the output state (called the "ansatz"). In the context of quantum simulation, very often the goal is to prepare ground states and the cost function is the expectation value of a Hamiltonian. Thus, if we define the reference (initial state) as $\lvert \psi\rangle$, the Hamiltonian as $H$ and the parameterized quantum circuit as $U(\vec{\theta})$ where $\vec{\theta}$ are the varaitional parameters, then the goal is to minimize the cost function
$$
E(\vec \theta) =  \langle \psi \rvert
U^\dagger(\vec{\theta}) H U(\vec{\theta})
\lvert \psi\rangle.
$$
A classical optimization algorithm can be used to find the $\vec{\theta}$ that minimizes the value of the expression. The performance of a variational algorithm depends crucially on the choice of ansatz circuit $U(\vec{\theta})$, the choice of reference, and the strategy for choosing the initial parameters $\vec{\theta}$ since typically global optimizing is challenging and one needs to begin reasonably close to the intended state. One possibility is to use an ansatz of the form
$$
U(\vec{\theta}) = \prod_j \exp(-i \theta_j H_j)
$$
where the $H = \sum_j H_j$. This ansatz is inspired by a low Trotter-number Trotter-Suzuki based approximation to adiabatic state preparation. OpenFermion-Cirq contains routines for constructing ansatzes of this form which use as templates the Trotter step algorithms implemented in the `trotter` module.

In this tutorial we will demonstrate the construction and optimization of a variational ansatz for a jellium Hamiltonian. We will use an ansatz based on the `LINEAR_SWAP_NETWORK` Trotter step, which takes as input a DiagonalCoulombHamiltonian.

In [1]:
import openfermion
import openfermioncirq

# Set parameters of jellium model.
wigner_seitz_radius = 5. # Radius per electron in Bohr radii.
n_dimensions = 2 # Number of spatial dimensions.
grid_length = 2 # Number of grid points in each dimension.
spinless = True # Whether to include spin degree of freedom or not.
n_electrons = 2 # Number of electrons.

# Figure out length scale based on Wigner-Seitz radius and construct a basis grid.
length_scale = openfermion.hamiltonians.wigner_seitz_length_scale(
    wigner_seitz_radius, n_electrons, n_dimensions)
grid = openfermion.Grid(n_dimensions, grid_length, length_scale)

# Initialize the model and compute its ground energy in the correct particle number manifold
fermion_hamiltonian = openfermion.jellium_model(grid, spinless=spinless, plane_wave=False)
hamiltonian_sparse = openfermion.get_sparse_operator(fermion_hamiltonian)
ground_energy, _ = openfermion.jw_get_ground_state_at_particle_number(
    hamiltonian_sparse, n_electrons)
print('The ground energy of the jellium Hamiltonian at {} electrons is {}'.format(
    n_electrons, ground_energy))

# Convert to DiagonalCoulombHamiltonian type.
hamiltonian = openfermion.get_diagonal_coulomb_hamiltonian(fermion_hamiltonian)

# Create a swap network Trotter ansatz.
iterations = 1  # This is the number of Trotter steps to use in the ansatz.
ansatz = openfermioncirq.SwapNetworkTrotterAnsatz(
    hamiltonian,
    iterations=iterations)

print('Created a variational ansatz with the following circuit:')
print(ansatz.circuit.to_text_diagram(transpose=True))

The ground energy of the jellium Hamiltonian at 2 electrons is -0.2697672439172564
Created a variational ansatz with the following circuit:
0    1         2      3
│    │         │      │
XXYY─XXYY^T0_1 XXYY───XXYY^T2_3
│    │         │      │
@────@^V0_1    @──────@^V2_3
│    │         │      │
×ᶠ───×ᶠ        ×ᶠ─────×ᶠ
│    │         │      │
│    @─────────@^V0_3 │
│    │         │      │
│    ×ᶠ────────×ᶠ     │
│    │         │      │
XXYY─XXYY^T1_3 XXYY───XXYY^T0_2
│    │         │      │
@────@^V1_3    @──────@^V0_2
│    │         │      │
×ᶠ───×ᶠ        ×ᶠ─────×ᶠ
│    │         │      │
Z^U3 @─────────@^V1_2 Z^U0
│    │         │      │
│    ×ᶠ────────×ᶠ     │
│    │         │      │
│    Z^U2      Z^U1   │
│    │         │      │
│    @─────────@^V1_2 │
│    │         │      │
│    ×ᶠ────────×ᶠ     │
│    │         │      │
@────@^V1_3    @──────@^V0_2
│    │         │      │
XXYY─XXYY^T1_3 XXYY───XXYY^T0_2
│    │         │      │
×ᶠ───×ᶠ        ×ᶠ─────×ᶠ
│    │         │      │

In the last lines above we instantiated a class called `SwapNetworkTrotterAnsatz` which inherits from the general `VariationalAnsatz` class in OpenFermion-Cirq. A VariationalAnsatz is essentially a parameterized circuit that one constructs so that parameters can be supplied symbolically. This way one does not (necessarily) need to recompile the circuit each time the variational parameters change.

Optimizing an ansatz requires the creation of a VariationalStudy object. A VariationalStudy encapsulates the evaluation of the objective function and stores the results of optimizations. It also includes an optional state preparation circuit to be applied prior to the ansatz circuit. For this example, we will prepare the initial state as an eigenstate of the one-body operator of the Hamiltonian. Since the one-body operator is a quadratic Hamiltonian, its eigenstates can be prepared using the `prepare_gaussian_state` method. We will use the HamiltonianVariationalStudy class, which stores a Hamiltonian and evaluates parameters by simulating the corresponding quantum circuit and computing the expectation value of the Hamiltonian on the final state. The SwapNetworkTrotterAnsatz class also includes a default setting of parameters which is inspired by the idea of state preparation by adiabatic evolution the mean-field state.

In [2]:
# Use preparation circuit for mean-field state
import cirq
preparation_circuit = cirq.Circuit.from_ops(
    openfermioncirq.prepare_gaussian_state(
        ansatz.qubits,
        openfermion.QuadraticHamiltonian(hamiltonian.one_body),
        occupied_orbitals=range(n_electrons)))

# Create a Hamiltonian variational study
study = openfermioncirq.HamiltonianVariationalStudy(
    'jellium_study',
    ansatz,
    hamiltonian=hamiltonian,
    preparation_circuit=preparation_circuit)

print("Created a variational study with {} qubits and {} parameters".format(
    len(study.ansatz.qubits), study.num_params))
print("The value of the objective with default initial parameters is {}".format(
    study.evaluate(study.default_initial_params())))
print("The circuit of the study is")
print(study.circuit.to_text_diagram(transpose=True))

Created a variational study with 4 qubits and 14 parameters
The value of the objective with default initial parameters is -0.19655859607069062
The circuit of the study is
0    1         2          3
│    │         │          │
X    X         │          │
│    │         │          │
│    YXXY──────#2^0.995   │
│    │         │          │
YXXY─#2^-0.502 Z^0.0      │
│    │         │          │
│    Z^0.0     YXXY───────#2^-0.498
│    │         │          │
│    YXXY──────#2^0.00482 Z^0.0
│    │         │          │
│    │         Z^0.0      │
│    │         │          │
XXYY─XXYY^T0_1 XXYY───────XXYY^T2_3
│    │         │          │
@────@^V0_1    @──────────@^V2_3
│    │         │          │
×ᶠ───×ᶠ        ×ᶠ─────────×ᶠ
│    │         │          │
│    @─────────@^V0_3     │
│    │         │          │
│    ×ᶠ────────×ᶠ         │
│    │         │          │
XXYY─XXYY^T1_3 XXYY───────XXYY^T0_2
│    │         │          │
@────@^V1_3    @──────────@^V0_2
│    │         │          │
×ᶠ───×

As we can see, our initial guess isn't particularly close to the target energy. Optimizing the study requires the creation of an OptimizationParams object. The most import component of this object is the optimization algorithm to use. OpenFermion-Cirq includes a wrapper around the the `minimize` method of Scipy's `optimize` module and more optimizers will be included in the future. Let's perform an optimization using the COBYLA method. Since this is just an example, we will set the maximum number of function evaluations to 100 so that it doesn't run too long.

In [3]:
# Perform an optimization run.
from openfermioncirq.optimization import ScipyOptimizationAlgorithm, OptimizationParams
algorithm = ScipyOptimizationAlgorithm(
    kwargs={'method': 'COBYLA'},
    options={'maxiter': 100},
    uses_bounds=False)
optimization_params = OptimizationParams(
    algorithm=algorithm,
    initial_guess=study.default_initial_params())
result = study.optimize(optimization_params)
print(result.optimal_value)

-0.2697256334414833


In practice, the expectation value of the Hamiltonian cannot be measured exactly due to errors from finite sampling. This manifests as an error, or noise, in the measured value of the energy which can be reduced at the cost of more measurements. The HamiltonianVariationalStudy class incorporates a realistic model of this noise (shot-noise). The OptimizationParams object can have a `cost_of_evaluate` parameter which in this case represents the number of measurements used to estimate the energy for a set of parameters. If we are interested in how well an optimizer performs in the presence of noise, then we may want to repeat the optimization several times and see how the results vary between repetitions.

Below, we will perform the same optimization, but this time using the noise model. We will allow one million measurements per energy evaluation and repeat the optimization three times. Since this time the function evaluations are noisy, we'll also indicate that the final parameters of the study should be reevaluated according to a noiseless simulation. Finally, we'll print out a summary of the study, which includes all results obtained so far (including from the previous cell).

In [5]:
optimization_params = OptimizationParams(
    algorithm=algorithm,
    initial_guess=study.default_initial_params(),
    cost_of_evaluate=1e6)
study.optimize(
    optimization_params,
    identifier='COBYLA with maxiter=100, noisy',
    repetitions=3,
    reevaluate_final_params=True,
    use_multiprocessing=True)
print(study.summary)

This study contains 2 results.
The optimal value found among all results is -0.2697256334414833.
It was found by the run with identifier 0.
Result details:
    Identifier: 0
        Optimal value: -0.2697256334414833
        Number of repetitions: 1
        Optimal value 1st, 2nd, 3rd quartiles:
            [-0.2697256334414833, -0.2697256334414833, -0.2697256334414833]
        Num evaluations 1st, 2nd, 3rd quartiles:
            [100.0, 100.0, 100.0]
        Cost spent 1st, 2nd, 3rd quartiles:
            [0.0, 0.0, 0.0]
    Identifier: COBYLA with maxiter=100, noisy
        Optimal value: -0.2686537961624912
        Number of repetitions: 3
        Optimal value 1st, 2nd, 3rd quartiles:
            [-0.26818290989782767, -0.26771202363316415, -0.26747783516586965]
        Num evaluations 1st, 2nd, 3rd quartiles:
            [100.0, 100.0, 100.0]
        Cost spent 1st, 2nd, 3rd quartiles:
            [100000000.0, 100000000.0, 100000000.0]


We see then that in the noisy study the optimizer fails to converge to the final result with high enough accuracy. Apparently then one needs more measurements, a more stable optimizer, or both!